# Earthquake Bot — Profit Dashboard

Источник истины: **Polymarket API** (on-chain баланс + positions + activity)

In [1]:
import json
import urllib.request
from datetime import datetime, timezone

In [2]:
# Получить текущий баланс USDC.e с блокчейна
import urllib.request

WALLET = '0xff36fc6De4CCDd290C14EE69244c21c1803Ad5b7'
USDC_E = '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'

# balanceOf(address) call
selector = '0x70a08231'
padded = WALLET[2:].lower().zfill(64)
call_data = selector + padded

RPCS = ['https://polygon-rpc.com', 'https://polygon-bor-rpc.publicnode.com', 'https://rpc.ankr.com/polygon']

current_balance = None
for rpc in RPCS:
    try:
        payload = json.dumps({
            'jsonrpc': '2.0', 'id': 1, 'method': 'eth_call',
            'params': [{'to': USDC_E, 'data': call_data}, 'latest']
        }).encode()
        req = urllib.request.Request(rpc, data=payload,
                                     headers={'Content-Type': 'application/json', 'User-Agent': 'Mozilla/5.0'})
        resp = json.loads(urllib.request.urlopen(req, timeout=10).read())
        current_balance = int(resp['result'], 16) / 1e6
        print(f'USDC.e balance: ${current_balance:.2f}  (via {rpc.split("//")[1].split("/")[0]})')
        break
    except Exception as e:
        print(f'{rpc}: {e}')

if current_balance is None:
    raise RuntimeError('Cannot fetch balance from any RPC')

https://polygon-rpc.com: HTTP Error 401: Unauthorized
USDC.e balance: $1000.80  (via polygon-bor-rpc.publicnode.com)


In [3]:
# Реальные депозиты (переводы с основного аккаунта)
DEPOSITS = [230.41, 99.50, 1044.98, 198.71, 249.94, 227.16]
total_deposits = sum(DEPOSITS)
print(f'Total deposits: ${total_deposits:.2f}')

Total deposits: $2050.70


In [4]:
# ══════════════════════════════════════════════════════════════
# POLYMARKET API — PROFIT DASHBOARD (источник истины)
# ══════════════════════════════════════════════════════════════

# --- 1. Все трейды с Polymarket ---
all_trades = []
offset = 0
while True:
    url = f'https://data-api.polymarket.com/activity?user={WALLET}&limit=100&offset={offset}'
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    batch = json.loads(urllib.request.urlopen(req, timeout=15).read())
    if not batch:
        break
    all_trades.extend(batch)
    offset += 100
    if len(batch) < 100:
        break

buys  = [t for t in all_trades if t.get('side') == 'BUY'  and t['type'] == 'TRADE']
sells = [t for t in all_trades if t.get('side') == 'SELL' and t['type'] == 'TRADE']
redeems = [t for t in all_trades if t['type'] == 'REDEEM']
rewards = [t for t in all_trades if t['type'] == 'REWARD']

total_bought  = sum(float(t['usdcSize']) for t in buys)
total_sold    = sum(float(t['usdcSize']) for t in sells)
total_redeemed = sum(float(t['usdcSize']) for t in redeems)
total_rewards  = sum(float(t['usdcSize']) for t in rewards)

# --- 2. Текущие позиции с Polymarket ---
url2 = f'https://data-api.polymarket.com/positions?user={WALLET}&sizeThreshold=0.1&limit=100'
req2 = urllib.request.Request(url2, headers={'User-Agent': 'Mozilla/5.0'})
pm_positions = json.loads(urllib.request.urlopen(req2, timeout=15).read())

pm_cost_basis   = sum(float(p['initialValue']) for p in pm_positions)
pm_market_value = sum(float(p['currentValue']) for p in pm_positions)

# --- 3. Таблица ---
print('═' * 55)
print('  POLYMARKET API — PROFIT DASHBOARD')
print('═' * 55)
print()
print(f'  Trades:  {len(buys)} buys, {len(sells)} sells, {len(redeems)} redeems, {len(rewards)} rewards')
print()
print(f'  ┌─────────────────────────────┬────────────┐')
print(f'  │ Total spent on buys         │ ${total_bought:>9.2f} │')
print(f'  │ Total received from sells   │ ${total_sold:>9.2f} │')
print(f'  │ Redemptions                 │ ${total_redeemed:>9.2f} │')
print(f'  │ Rewards (liquidity)         │ ${total_rewards:>9.2f} │')
print(f'  ├─────────────────────────────┼────────────┤')
net = total_sold + total_redeemed + total_rewards - total_bought
print(f'  │ Net trading flow            │ ${net:>+9.2f} │')
print(f'  └─────────────────────────────┴────────────┘')
print()
print(f'  ┌─────────────────────────────┬────────────┐')
print(f'  │ Deposits (real transfers)   │ ${total_deposits:>9.2f} │')
print(f'  │ USDC.e balance (on-chain)   │ ${current_balance:>9.2f} │')
print(f'  │ Positions cost basis (PM)   │ ${pm_cost_basis:>9.2f} │')
print(f'  │ Positions market value (PM) │ ${pm_market_value:>9.2f} │')
print(f'  ├─────────────────────────────┼────────────┤')
profit_cost   = current_balance + pm_cost_basis - total_deposits
profit_market = current_balance + pm_market_value - total_deposits
print(f'  │ PROFIT (at cost basis)      │ ${profit_cost:>+9.2f} │')
print(f'  │ PROFIT (at market value)    │ ${profit_market:>+9.2f} │')
pct = profit_market / total_deposits * 100
print(f'  │ ROI                         │ {pct:>+8.1f}%  │')
print(f'  └─────────────────────────────┴────────────┘')

# Deposit verification
implied_deposits = current_balance + total_bought - total_sold - total_redeemed - total_rewards
deposit_diff = total_deposits - implied_deposits
print()
if abs(deposit_diff) < 5:
    print(f'  ✓ Deposits verified (diff ${deposit_diff:+.2f})')
else:
    print(f'  ⚠ Deposit mismatch: real ${total_deposits:.2f} vs implied ${implied_deposits:.2f} (diff ${deposit_diff:+.2f})')

═══════════════════════════════════════════════════════
  POLYMARKET API — PROFIT DASHBOARD
═══════════════════════════════════════════════════════

  Trades:  355 buys, 169 sells, 2 redeems, 5 rewards

  ┌─────────────────────────────┬────────────┐
  │ Total spent on buys         │ $  4771.02 │
  │ Total received from sells   │ $  3506.75 │
  │ Redemptions                 │ $     1.75 │
  │ Rewards (liquidity)         │ $     5.98 │
  ├─────────────────────────────┼────────────┤
  │ Net trading flow            │ $ -1256.55 │
  └─────────────────────────────┴────────────┘

  ┌─────────────────────────────┬────────────┐
  │ Deposits (real transfers)   │ $  2050.70 │
  │ USDC.e balance (on-chain)   │ $  1000.80 │
  │ Positions cost basis (PM)   │ $  1460.72 │
  │ Positions market value (PM) │ $  1473.76 │
  ├─────────────────────────────┼────────────┤
  │ PROFIT (at cost basis)      │ $  +410.82 │
  │ PROFIT (at market value)    │ $  +423.86 │
  │ ROI                         │    +20.7% 